In [2]:
import numpy as np
import matplotlib.pyplot as plt
import rdkit 
import cclib
import itertools
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import requests
from itertools import combinations
from typing import List

In [3]:
#from aiida.engine import ToContext, WorkChain, calcfunction
#from aiida.plugins.factories import CalculationFactory

In [4]:
import os
import os.path
import shutil
import subprocess

In [5]:
def fragment_dictionary(name_of_smi_file):
    #loads the smile file
    smile_list = np.genfromtxt(f'{name_of_smi_file}.smi',dtype='str')
    #creates a dictionary of fragments
    fragment_dic = { i : d for i, d in enumerate(smile_list)}
    return fragment_dic

def combined_fragments_dic(dic):
    # Get all SMILES for all possible combinations (of 2 fragments) of the fragment list
    #All the combinations of dimers smiles from fragment dictionary
    combo_smiles = (f'{v1}.{v2}'.replace('(*)', '9')
                    for v1, v2 in itertools.combinations(dic.values(), r=2))
    #All the combinations of dimer names from fragment dictionary
    combo_names = [(f'{k1}-{k2}')
                    for k1, k2 in itertools.combinations(dic.keys(), r=2)]
    
        # Convert all SMILES to mol objects
    #mols = [Chem.MolFromSmiles(smi) for smi in combo_smiles]
    
    #relating the dimer names and dimer smiles back to one another
    combo_dic = { k : v for k, v in zip(combo_names, combo_smiles) }
        
    return combo_dic
    #Creates a list of all the possible combinations as RDKit info
    
def combined_fragments_dic_mol(dic):
    # Get all SMILES for all possible combinations (of 2 fragments) of the fragment list
    #All the combinations of dimers smiles from fragment dictionary
    combo_smiles = (f'{v1}.{v2}'.replace('(*)', '9')
                    for v1, v2 in itertools.combinations(dic.values(), r=2))
    #All the combinations of dimer names from fragment dictionary
    combo_names = [(f'{k1}-{k2}')
                    for k1, k2 in itertools.combinations(dic.keys(), r=2)]
    
        # Convert all SMILES to mol objects
    mols = [Chem.MolFromSmiles(smi) for smi in combo_smiles]
    
    #relating the dimer names and dimer smiles back to one another
    combo_dic = { k : v for k, v in zip(combo_names, mols) }
        
    return combo_dic
    #Creates a list of all the possible combinations as RDKit info
    
def getBond(mol):
    #rotatable bonds
    pattern = Chem.MolFromSmarts('[R!$(*#*)&!D1]-!@[R!$(*#*)&!D1]')
    bonds = mol.GetSubstructMatches(pattern)
    return bonds
        
#3 14 21 22 S 35 10.0
def getTorsion(mol,bond):
    # get neighbors of first atom in bond
    for atom in mol.GetAtomWithIdx(bond[0]).GetNeighbors(): 
        idx = atom.GetIdx()
        mass = atom.GetMass() 
        if idx!=bond[1]: #excludes the other atom in the bond
            if mass > 13: # N, S, O get priority
                first=idx
                break
            if mass > 12: # otherwise C 
                first=idx
    # get neighbors of second atom in bond
    for atom in mol.GetAtomWithIdx(bond[1]).GetNeighbors():
        idx=atom.GetIdx()
        mass = atom.GetMass() 
        if idx!=bond[0]: #excludes the other atom in the bond
            if mass > 13: # N, S, O get priority
                last=idx
                break
            if mass > 12: # otherwise C 
                last=idx

    return (first, bond[0], bond[1], last)
        
def embed_molecule(mol):
    addhs = Chem.AddHs(mol)
    AllChem.EmbedMolecule(addhs) # the embedded molecule
    return addhs
        
def write_guassian(job_type, dimer_name, dimer=None, torsion=None, conformer=None):
    """
    job_type : This is determined by what calculations you want to happen. The options are
                'torsional scan', 'neutral population', 'opt anion', 'ver anion', 'opt cation'
                and 'ver cation'
    
    dimer_name : the name of the dimer from the dictionary e.g. if fragment 0 was attached to fragment 1
                    then the dimer name is 0-1
                    
    dimer : The rdkit string of the dimer
    
    torsion : The getTorsion information in regards to the rotatable bond
    
    conformer : The corrected rdkit coordinates of a dimer, only used after a torsional scan has been done and
                is the lowest energy coordinates
    """
    
    if (job_type=='torsional scan'):
        
        # get atom names
        symbols = [a.GetSymbol() for a in dimer.GetAtoms()] 
        # get x y z coords
        geometry=dimer.GetConformers()[0]
        # unpack torsion terms and add 1 for fortran 1-based numbering
        a1, a2, a3, a4 = [t+1 for t in torsion]
        torsion_data = f'{a1} {a2} {a3} {a4} S 35 10.0\n'
        file_name = f'{dimer_name}_torsion.com' 
        chk_name = f'{dimer_name}_torsion.chk' 
        title = f'scan dihedral {dimer_name}'
        calculation = 'opt=modredundant'
        mult_chg = '0 1' # by default all dimers are neutral and singlets!
        
    if (job_type=='neutral population'):

        # get atom names
        symbols = [a.GetSymbol() for a in dimer.GetAtoms()] 
        # get x y z coords
        geometry = conformer
        torsion_data = f' \n'
        file_name = f'{dimer_name}_neu_opt_pop.com' 
        chk_name = f'{dimer_name}_neu_opt_pop.chk' 
        title = f'Neutral optimised with HOMO/LUMO {dimer_name}'
        calculation = 'opt=modredundant, Pop=Full'
        mult_chg = '0 1' # by default all dimers are neutral and singlets!
        
    if (job_type=='opt anion'):
        
        # get atom names
        symbols = [a.GetSymbol() for a in dimer.GetAtoms()]
        geometry = conformer
        torsion_data = f' \n'
        file_name = f'{dimer_name}_opt_anion.com' 
        chk_name = f'{dimer_name}_opt_anion.chk' 
        title = f'Optimisation of anion dimer {dimer_name}'
        calculation = 'opt=modredundant'
        mult_chg = '-1 2' # This is a negative dimer calc so multiplicty and charge change!
        
    if (job_type=='ver anion'):
        
        # get atom names
        symbols = [a.GetSymbol() for a in dimer.GetAtoms()]
        geometry = conformer
        torsion_data = f' \n'
        file_name = f'{dimer_name}_ver_anion.com' 
        chk_name = f'{dimer_name}_ver_anion.chk' 
        title = f'Veritcal of anion dimer {dimer_name}'
        calculation = 'SP'
        mult_chg = '-1 2' # This is a negative dimer calc so multiplicty and charge change!
        
    if (job_type=='opt cation'):
        
        # get atom names
        symbols = [a.GetSymbol() for a in dimer.GetAtoms()]
        geometry = conformer
        torsion_data = f' \n'
        file_name = f'{dimer_name}_opt_cat.com' 
        chk_name = f'{dimer_name}_opt_cat.chk' 
        title = f'Optimisation of cation dimer {dimer_name}'
        calculation = 'opt=modredundant'
        mult_chg = '1 2' # This is a negative dimer calc so multiplicty and charge change!
        
    if (job_type=='ver cation'):
        
        # get atom names
        symbols = [a.GetSymbol() for a in dimer.GetAtoms()]
        geometry = conformer
        torsion_data = f' \n'
        file_name = f'{dimer_name}_ver_cat.com' 
        chk_name = f'{dimer_name}_ver_cat.chk' 
        title = f'Vertical of cation dimer {dimer_name}'
        calculation = 'SP'
        mult_chg = '1 2' # This is a negative dimer calc so multiplicty and charge change!
        
    
    with open(file_name, 'w') as file:
        file.write(f'%chk={chk_name}\n') # 
        file.write(f'#p B3LYP/6-31G* {calculation}, nosymm\n') # 
        file.write(' \n')# 
        file.write(f'{title}\n')# 
        file.write(' \n')# 
        file.write(f'{mult_chg} \n')# 
        for atom,symbol in enumerate(symbols):
            p = geometry.GetAtomPosition(atom)
            # atom  x y z
            line = f' {symbol} {p.x:.5f} {p.y:.5f} {p.z:.5f} \n' 
            file.write(line)
        file.write(' \n')
        file.write(torsion_data)
        file.write(' \n')

def write_slurm(job_name):
    
    file_name = f'{job_name}.sh'
    title = f'#!/bin/bash --login'
    with open(file_name, 'w') as file:
        file.write(f'{title}\n')# 
        file.write(f'#SBATCH -o g09_%J.out \n')
        file.write(f'#SBATCH -e g09_%J.err \n')#
        file.write(f'#SBATCH --job-name={job_name} \n')
        file.write(f'#SBATCH -p cpu \n')
        file.write(f'#SBATCH --ntasks=10 \n')
        file.write(f'#SBATCH --nodes=1 \n')
        file.write(f'#SBATCH --tasks-per-node=10 \n')
        file.write(f'#SBATCH --mem-per-cpu=4000 \n')
        file.write(f'#SBATCH --time=48:00:00 \n')
        file.write(' \n')#
        file.write(f'INPUTFILE={job_name}.com \n')
        file.write(f'OUTPUTFILE="\$(basename "\$INPUTFILE" .com).log" \n')
        file.write(' \n')
        file.write(f'module purge \n')
        file.write(f'module load test_switch_kcl/1.0.0-gcc-9.4.0 \n')
        file.write(f'module load gaussian_sse4_kcl/09-E-gcc-9.4.0 \n')
        file.write(f'export GOMP_CPU_AFFINITY=$SGE_BINDING \n')
        file.write(f'export KMP_AFFINITY="explicit,proclist=$SGE_BINDING,verbose" \n')
        file.write(f'#source $g09root/bsd/g09.login \n')
        file.write(' \n')
        file.write(f'echo "G09 job \$SLURM_JOBID" \n')
        file.write(f'echo "INPUT \$INPUTFILE" \n')
        file.write(f'echo "OUTPUT \$OUTPUTFILE" \n')
        file.write(f'echo "Running \$SLURM_NTASKS on \$SLURM_JOB_NODELIST" \n')
        file.write(' \n')
        file.write(f'#Execution Line \n')
        file.write(f'g09 "$INPUTFILE" > "$OUTPUTFILE" \n')
        
def submit_slurm_job(dimer_name):
    """
    Submit a slurm job

    """
    
    string = f'sbatch {dimer_name}.sh' 
    
    process = subprocess.run(string,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.PIPE, shell=True,check=True)
    return process.stdout

def delete_sh(dimer_name):
    return os.remove(f'{dimer_name}.sh')

def delete_com(dimer_name):
    return os.remove(f'{dimer_name}.com')

In [6]:
def TortionalScanAuto(smiles_list_with_attach):
    
    #loads
    mol_list = np.genfromtxt(smiles_list_with_attach ,dtype='str')
    #function together
    mol_num = { i : d for i, d in enumerate(mol_list)}
    
    #returns dictionary with name of dimer and SMILE combined string
    dimers_named = combine_fragments_dic(mol_num)
    
    #function to create rdkit molecule
    #for each dimer find the bond, find the torsion, write the file

    bonds_list = []
    for dimer in dimers_list:
        bonds_list.append(getBond(dimer)) #finds the bonds between the fragments

    torsions_list=[]
    for bond, dimer in zip(bonds_list, dimers_list):
        torsions_list.append(getTorsion(dimer,list(bond[0]))) #obtains the torsion of the bond between fragments
    
    dimers_3dlist=[]
    for dimer in dimers_list:
        dimer = embed_molecule(dimer)
        dimers_3dlist.append(dimer) #get coordinates of the dimer

    dimer_dic = { i : d for i, d in enumerate(dimers_3dlist)} #creates a list and numbers all the dimers
    
    os.mkdir('tortional_inputs') #creates a directory named tortional_scan
    
    os.chdir('tortional_inputs') #goes into that directory
    
    for (k, v), t in zip(dimer_dic.items(), torsions_list):
        os.mkdir(f'{k}-Job')
        os.chdir(f'{k}-Job')
        write_gaussian('torsional scan', k, v, t) #writes the input file
        write_slurm_input('torsional scan', k) #writes the SLURM File
        #submit_slurm_job(k) #submits SLURM jobs
        os.chdir(os.path.dirname(os.getcwd())) #goes back to previous directory
        #delete_sh(k) #deletes all the SLURM files
        #delete_com(k) #deletes all the inputs

In [21]:
def torsional_scan_run(example_mols_pres):
    
    #Loads the smi file and creates a dictionary of fragments
    farg_dic = fragment_dictionary('example_mols_pres')
    #Creates a dictionary of all the dimers from the fragment dictionary
    dimers_dic = combined_fragments_dic(farg_dic)
    
    #creates a directory named tortional_scan
    os.mkdir('tortional_inputs')
    
    #goes into that directory
    os.chdir('tortional_inputs')
    
    #torsion_dic, bond_dic, mol_dic = {}
    for dimer_name, dimer_smiles in dimers_dic.items():
        #makes directory called after the job name
        os.mkdir(f'{dimer_name}-job')
        #goes into that directory
        os.chdir(f'{dimer_name}-job')
        
        #turns smiles string into rdkit object
        mol = Chem.MolFromSmiles(dimer_smiles)
        #finds the bond between the fragment
        bond = getBond(mol)
        #torsion of the bond between the fragment
        torsion = getTorsion(mol, bond[0])
        #gets rdkit estimated coordinates of dimer
        mold3d = embed_molecule(mol)
        
        #writes a guassian input file
        write_guassian('torsional scan', dimer_name, mold3d, torsion)
        #writes the slurm file
        write_slurm_input(job_name)
        #submits the slurm jon
        #submit_slurm_job(dimer_name)
        #goes back to previous directory
        os.chdir(os.path.dirname(os.getcwd()))

In [29]:
torsional_scan_run('example_mols_pres')

In [15]:
dic = fragment_dictionary('example_mols_pres')

In [16]:
combined_fragments_dic(dic)

{'0-1': 'CC1=C9SC=C1.COC1=C9SC=C1',
 '0-2': 'CC1=C9SC=C1.CNC1=C9SC=C1',
 '1-2': 'COC1=C9SC=C1.CNC1=C9SC=C1'}

In [30]:
os.chdir('/Users/student/workflow/')

In [25]:
os.getcwd()

'/Users/student/workflow'